In [1]:
import numpy as np
import matplotlib.pylab as plt
import scipy.io
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers
from tensorflow import keras
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
import absl.logging
import tensorflow as tf


def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))

absl.logging.set_verbosity(absl.logging.ERROR) # .h5 에러메세지 무시

mat_file = scipy.io.loadmat("data/PTSDvsHC.mat")

dataset = mat_file['dataset'] 
# 90개의 데이터, 62개의 전극 채널, 1001의 타임스텝 - 1001씩 이미 에포킹을 한 상태이며, 그에따른 평균을 낸 것임(에포킹에 대해)

subject_test_acc = []

X = dataset
y = mat_file['Y']
y = to_categorical(y, num_classes = 2)

X = X.reshape(90, 1001, 62, 1)

input_shape = (1001, 62, 1)

for i in range(len(dataset)): # loso
    X_train = np.concatenate([X[:i], X[i+1:]], axis=0)
    X_test = X[i:i+1]
    y_train = np.concatenate([y[:i], y[i+1:]], axis=0)
    y_test = y[i:i+1]

    # print(f"X_train_shape : {X_train.shape}")
    # print(f"X_test_shape : {X_test.shape}")
    # print(f"y_train_shape : {y_train.shape}")
    # print(f"y_test_shape : {y_test.shape}")
    
    model_input = layers.Input(shape = input_shape)
    br1 = layers.Conv2D(32, (200, 1), padding = 'same')(model_input) # temporal filter
    br1 = layers.Activation(mish)(br1) # result1_matrix, shape=(1001, 62, 32)
    # print(br1.shape)

    input_shape_tr = layers.Permute((2, 1, 3))(model_input) # (62, 1001, 1)
    br2 = layers.Conv2D(32, (62, 1), padding = 'same')(input_shape_tr)
    br2 = layers.Activation(mish)(br2) # result2_matrix, shape=(62, 1001, 32)

    # br1, br2 matrix multiply
    br1 = layers.Permute((3, 1, 2))(br1) # (32, 1001, 62)
    br2 = layers.Permute((3, 1, 2))(br2) # (32, 62, 1001)
    br_mul = layers.Lambda(lambda x: tf.matmul(x[0], x[1]))([br2, br1])
    # print(br_mul.shape)  # (32, 62, 62)

    # branch matrix multiply convolution
    br_mul = layers.Permute((2, 3, 1))(br_mul) # (62, 62, 32) <- 풀링하기 위함
    print(br_mul)
    br_mul = layers.MaxPooling2D(pool_size=(62, 62))(br_mul) # (62, 62)
    br_mul = layers.Conv2D(32, (3, 3), padding = 'same')(br_mul)
    br_mul = layers.Activation(mish)(br_mul)

    end = layers.Flatten()(br_mul)

    # globalmaxpooling을 할 경우, (batch_size, channel)로 output이 나오기 때문에 사용하지 않기로 함
    # maxpooling 사용

    prediction = layers.Dense(2, activation = 'softmax')(end)
    model = keras.Model(inputs = model_input, outputs = prediction)
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    history = model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs=100, batch_size=1, verbose=0)
    predictions = model.predict(X_test)
    
    predicted_classes = np.argmax(predictions, axis=1)
    actual_classes = np.argmax(y_test, axis=1)

    acc = accuracy_score(predicted_classes, actual_classes)

    # model.summary()
    
    print(f"fold {i} accuracy : {acc}")

    subject_test_acc.append(acc)
print(f"loso mean accuracy : {np.mean(subject_test_acc)}")


<KerasTensor shape=(None, 62, 62, 32), dtype=float32, sparse=False, name=keras_tensor_12>
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
fold 0 accuracy : 1.0
<KerasTensor shape=(None, 62, 62, 32), dtype=float32, sparse=False, name=keras_tensor_30>
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
fold 1 accuracy : 1.0
<KerasTensor shape=(None, 62, 62, 32), dtype=float32, sparse=False, name=keras_tensor_48>
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
fold 2 accuracy : 0.0
<KerasTensor shape=(None, 62, 62, 32), dtype=float32, sparse=False, name=keras_tensor_66>
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
fold 3 accuracy : 1.0
<KerasTensor shape=(None, 62, 62, 32), dtype=float32, sparse=False, name=keras_tensor_84>
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
fold 4 accuracy : 1.0
<KerasTensor shape=(None, 62, 62, 32), dtype=float32, sparse=False, name=keras_tensor_102>
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
fold 5 accuracy : 1.0
<KerasTensor shape=(None, 62, 62, 32), dtype=float32, sparse=False, name=keras_tensor_120>
1